# Binary Tree Sequence to Newick Converter

This notebook converts binary tree sequence files (`.trees`) generated by Slim simulations into Newick format (`.tree` files) for downstream analysis. 

**What it does:**
- Reads binary tree sequence files from the `trees/` directory
- Converts each tree sequence to Newick format (one tree per line)
- Generates CSV files with genomic interval breakpoints for each tree
- Applies time jittering to ensure proper tree ordering

**Important:** This notebook should be run **once** after all simulations are complete, as it processes all tree sequence files in batch.


##

In [1]:
import os
import numpy as np
import tskit
import csv

# ─────────────────────────────────────────────────────────────────────────────
# Helper: jitter node times within each generation (skip samples)
# ─────────────────────────────────────────────────────────────────────────────
def jitter_within_generations(ts: tskit.TreeSequence, eps: float = 1e-9) -> tskit.TreeSequence:
    """
    Make node times strictly increasing within each generation by adding tiny,
    deterministic jitters to nodes that share the same time. Samples (time=0)
    are not jittered. Ordering is by node ID to be deterministic.
    """
    tables = ts.dump_tables()  # safe, editable copy
    times = tables.nodes.time.astype(float).copy()

    # Identify samples so we don't move them off time 0
    is_sample = np.zeros(len(times), dtype=bool)
    is_sample[ts.samples()] = True

    # For each unique time, spread tied non-sample nodes by eps * rank
    for t in np.unique(times):
        idx = np.where(times == t)[0]
        idx = [i for i in idx if not is_sample[i]]
        if len(idx) > 1:
            for rank, node_id in enumerate(sorted(idx)):
                times[node_id] = t + eps * rank

    # Write back and re-sort to keep table invariants
    tables.nodes.set_columns(
        flags=tables.nodes.flags,
        time=times,
        population=tables.nodes.population,
        individual=tables.nodes.individual,
        metadata=tables.nodes.metadata,
        metadata_offset=tables.nodes.metadata_offset,
    )
    tables.sort()
    return tables.tree_sequence()

# ─────────────────────────────────────────────────────────────────────────────
# Main conversion script: export .tree (Newick) + .csv (breakpoints)
# ─────────────────────────────────────────────────────────────────────────────
input_dir = "trees"
output_dir = "trees_newick"
os.makedirs(output_dir, exist_ok=True)

for filename in sorted(os.listdir(input_dir)):
    if filename.endswith(".trees") and filename.startswith("r"):
        input_path = os.path.join(input_dir, filename)
        stem = os.path.splitext(filename)[0]  # e.g. "r3000001"

        # Output files (.tree + .csv)
        output_tree_path = os.path.join(output_dir, f"{stem}.tree")
        output_csv_path  = os.path.join(output_dir, f"{stem}_breaks.csv")

        # Load, jitter, and convert to Newick
        ts = tskit.load(input_path)
        ts = jitter_within_generations(ts, eps=1e-9)

        # Export Newick file (one tree per line)
        with open(output_tree_path, "w") as f:
            for tree in ts.trees():
                f.write(tree.newick() + "\n")

        # Export CSV file with genomic intervals
        with open(output_csv_path, "w", newline="") as cf:
            writer = csv.writer(cf)
            writer.writerow(["file", "tree_index", "left", "right", "mid"])
            for i, tree in enumerate(ts.trees()):
                left, right = tree.interval.left, tree.interval.right
                mid = 0.5 * (left + right)
                writer.writerow([stem, i, left, right, mid])

        print(f"✅ {filename} → {output_tree_path}, {output_csv_path}")

print("🎉 All tree files converted successfully with jitter + interval CSVs!")

✅ r3000000.trees → trees_newick/r3000000.tree, trees_newick/r3000000_breaks.csv
✅ r3000001.trees → trees_newick/r3000001.tree, trees_newick/r3000001_breaks.csv
✅ r3000002.trees → trees_newick/r3000002.tree, trees_newick/r3000002_breaks.csv
✅ r3000003.trees → trees_newick/r3000003.tree, trees_newick/r3000003_breaks.csv
✅ r3000004.trees → trees_newick/r3000004.tree, trees_newick/r3000004_breaks.csv
✅ r3000005.trees → trees_newick/r3000005.tree, trees_newick/r3000005_breaks.csv
✅ r3000006.trees → trees_newick/r3000006.tree, trees_newick/r3000006_breaks.csv
✅ r3000007.trees → trees_newick/r3000007.tree, trees_newick/r3000007_breaks.csv
✅ r3000008.trees → trees_newick/r3000008.tree, trees_newick/r3000008_breaks.csv
✅ r3000009.trees → trees_newick/r3000009.tree, trees_newick/r3000009_breaks.csv
✅ r3000010.trees → trees_newick/r3000010.tree, trees_newick/r3000010_breaks.csv
✅ r3000011.trees → trees_newick/r3000011.tree, trees_newick/r3000011_breaks.csv
✅ r3000012.trees → trees_newick/r3000012

Ignore the following code

In [ ]:
# from pathlib import Path
# from pickle import FALSE
# import re

# # === set your folder here ===
# folder = Path("trees_newick")   # e.g., Path("trees")

# # Match: r300 + <digits> + <anything else>
# # Examples: r3000001.trees -> r3000251.trees
# #           r3007.txt      -> r300257.txt
# pat = re.compile(r"^(r300)(\d+)(.*)$")

# dry_run = False  # set to False to actually rename

# for p in folder.iterdir():
#     if not p.is_file():
#         continue
#     m = pat.match(p.name)
#     if not m:
#         continue

#     prefix, num_s, suffix = m.groups()
#     width = len(num_s)                 # preserve zero padding
#     new_num = int(num_s) - 250
#     new_name = f"{prefix}{str(new_num).zfill(width)}{suffix}"
#     target = p.with_name(new_name)

#     if target.exists():
#         print(f"SKIP (target exists): {p.name} -> {new_name}")
#         continue

#     print(f"{p.name} -> {new_name}")
#     if not dry_run:
#         p.rename(target)

r3000262_breaks.csv -> r3000012_breaks.csv
r3000365.tree -> r3000115.tree
r3000255_breaks.csv -> r3000005_breaks.csv
r3000414_breaks.csv -> r3000164_breaks.csv
r3000376_breaks.csv -> r3000126_breaks.csv
r3000341_breaks.csv -> r3000091_breaks.csv
r3000423_breaks.csv -> r3000173_breaks.csv
r3000298.tree -> r3000048.tree
r3000332.tree -> r3000082.tree
r3000277.tree -> r3000027.tree
r3000314_breaks.csv -> r3000064_breaks.csv
r3000476_breaks.csv -> r3000226_breaks.csv
r3000441_breaks.csv -> r3000191_breaks.csv
r3000323_breaks.csv -> r3000073_breaks.csv
r3000324.tree -> r3000074.tree
r3000261.tree -> r3000011.tree
r3000489_breaks.csv -> r3000239_breaks.csv
r3000373.tree -> r3000123.tree
r3000389_breaks.csv -> r3000139_breaks.csv
r3000408_breaks.csv -> r3000158_breaks.csv
r3000308.tree -> r3000058.tree
r3000471.tree -> r3000221.tree
r3000308_breaks.csv -> r3000058_breaks.csv
r3000426.tree -> r3000176.tree
r3000349.tree -> r3000099.tree
r3000430.tree -> r3000180.tree
r3000495_breaks.csv -> r30